In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os
import time

In [2]:
commersul = 'https://www.commersul.com.br/shop/'
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [3]:
query = pd.read_csv('produtos_bot_yury_not_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [4]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [5]:
def findOEM(text):
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) >= 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [6]:
def commersulSearch(nav, cod, index):
    nav.get(str(commersul))
    elemento = nav.find_element(By.XPATH, '//*[@id="th-search-form-1"]')
    elemento.send_keys(str(cod))
    elemento.send_keys(Keys.ENTER)
    nav.implicitly_wait(0.8)
    try:
        keys = []
        values = []
        keys.append('Nome: ')
        values.append(nav.find_element(By.CLASS_NAME, 'product_title').text)
        keys.append('Cod. Cotripal: ')
        values.append(index)
        
        elemento = nav.find_element(By.CLASS_NAME, 'woocommerce-product-gallery__wrapper')
        elemento = elemento.find_element(By.TAG_NAME, 'a')
        keys.append('Imagem: ')
        values.append(elemento.get_attribute('href'))
        
        elemento = nav.find_element(By.CLASS_NAME, 'th-product__meta')
        ths = elemento.find_elements(By.TAG_NAME, 'th')
        tds = elemento.find_elements(By.TAG_NAME, 'td')
        for th in ths:
            th = th.text
            keys.append(th)
        for td in tds:
            td = td.text
            values.append(td)
        dict_infos = dict(zip(keys, values))
        dict_infos.pop('COD. COMMERSUL')
        dict_infos = str(dict_infos)
        dict_infos = dict_infos.replace("'", '"')
        with open(f'{cwd}/Commersul/prods.json', 'a') as arquivo:
            arquivo.write(dict_infos)
    except Exception as e:
        notFound.loc[len(notFound)] = [index, 'Não encontrado no site']
        return

In [7]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)

In [8]:
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findOEM(cod)
    for item in cods:
        commersulSearch(nav, item, ID)

In [9]:
# commersulSearch(nav, '046781', 1)

In [11]:
display(notFound)

,Produto,Motivo
0,101011714,Não encontrado no site
1,100276657,Não encontrado no site
2,101066264,Não encontrado no site
3,100096446,Não encontrado no site
4,100185867,Não encontrado no site
...,...,...
101,105001070,Não encontrado no site
102,101167114,Não encontrado no site
103,101167114,Não encontrado no site
104,101053066,Não encontrado no site


In [12]:
notFound.to_excel(f'{cwd}/Commersul/notFound.xlsx')